<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center">
  <a href="https://docs.vlm.run"><b>Website</b></a> | 
  <a href="https://docs.vlm.run/"><b>API Docs</b></a> | 
  <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | 
  <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | 
  <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - Student Term Paper Review & Feedback


This comprehensive cookbook demonstrates [VLM Run Orion's](https://vlm.run/orion) to automatically review student term papers (PDFs or images) and provide actionable, structured feedback. Whether you're an educator, TA, or student seeking self-review, Orion can evaluate clarity, argument strength, citation integrity, grammar, and formatting—all with page-level precision.

For this notebook, we'll cover how to use the **VLM Run Agent Chat Completions API**—an OpenAI-compatible interface that supports image and data inputs alongside text—to generate plots directly from structured data.

We'll cover the following topics:
1. **Document Ingestion** – Upload a student paper via URL

2. **Academic Integrity Check** – Detect missing citations, weak sources, or plagiarism signals

3. **Writing Quality Assessment** – Evaluate grammar, tone, structure, and coherence

4. **Constructive Suggestions** – Generate revision tips tailored to academic standards

5. **Structured Output** – Return machine-readable feedback for LMS integration

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra: `pip install "vlmrun[openai]"`

## Setup

First, install the required packages and configure the environment.

In [7]:
# Install required packages
!pip install vlmrun[openai] --upgrade --quiet
!pip install pillow requests numpy --quiet


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [8]:
import os
import getpass
import json
from typing import List, Any
from functools import cached_property

import numpy as np
from PIL import Image
from pydantic import BaseModel, Field

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")

## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.

In [9]:
from vlmrun.client import VLMRun

client = VLMRun(
    api_key=VLMRUN_API_KEY, base_url="https://agent.vlm.run/v1", timeout=1000
)
print("VLM Run client initialized successfully!")
print(f"Base URL: https://agent.vlm.run/v1")
print(f"Model: vlmrun-orion-1")

VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1
Model: vlmrun-orion-1


## Response Models (dtypes)

We define structured Pydantic models to capture proofreading feedback with full traceability.



In [10]:
class FeedbackItem(BaseModel):
    category: str = Field(..., description="One of: 'grammar', 'clarity', 'argument', 'citation', 'formatting', 'originality'")
    page_number: int = Field(..., description="Page where issue occurs")
    excerpt: str = Field(..., description="Short quoted text")
    severity: str = Field(..., description="'low', 'medium', 'high'")
    suggestion: str = Field(..., description="Actionable revision advice")
    rationale: str = Field(..., description="Why this matters academically")

class TermPaperReview(BaseModel):
    total_feedback_items: int
    feedback_by_category: dict[str, int]
    feedback: List[FeedbackItem]
    overall_assessment: str = Field(..., description="Summary for instructor or student")
    grade_estimate: str = Field(..., description="e.g., 'A-', 'B+', 'Needs Major Revision'")

    def __repr__(self):
        cats = ', '.join(f"{k}={v}" for k, v in self.feedback_by_category.items())
        return f"TermPaperReview(grade='{self.grade_estimate}', items={self.total_feedback_items}, categories=[{cats}])"

print("✅ Proofreading response models defined!")

✅ Proofreading response models defined!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.

In [11]:
import hashlib
import cachetools
from typing import Type, TypeVar
from IPython.display import HTML
from vlmrun.common.image import encode_image


T = TypeVar('T', bound=BaseModel)


def display(images: Image.Image | list[Image.Image], texts: list[str] | None = None, width: int = 300):
    if isinstance(images, Image.Image):
        images = [images]
    if texts == None:
        texts = [None] * len(images)
    elif isinstance(texts, str):
        texts = [texts]
    elif len(texts) != len(images):
        raise ValueError("`texts` must be a list of the same length as `images`")

    imgs_html = ""
    for image, text in zip(images, texts):
        W, H = image.size
        if W > width:
            H = int(H * width / W)
            W = width
            image = image.resize((W, H))
        im_bytes = encode_image(image, format="JPEG")
        imgs_html += f"<div style='display:inline-block; margin:5px; text-align:center'>"
        imgs_html += f"<img src='{im_bytes}' style='width:{width}px; border-radius:6px'>"
        if text:
            imgs_html += f"<div style='font-size:12px; color:#666; margin-top:5px'>{text}</div>"
        imgs_html += f"</div>"
    return HTML(f"<div style='display:flex; flex-wrap:wrap'>{imgs_html}</div>")


def custom_key(prompt: str, images: list[Image.Image] | list[str] | None = None, doc: list[str] | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    image_keys = []
    if images:
        for image in images:
            if isinstance(image, Image.Image):
                thumb = image.copy()
                thumb.thumbnail((128, 128))
                encoded = encode_image(thumb, format="JPEG")
                image_keys.append(encoded)
            elif isinstance(image, str):
                image_keys.append(image)

    doc_keys = []
    if doc:
        if isinstance(doc, str):
            doc_keys.append(doc)
        elif isinstance(doc, list):
            for d_url in doc:
                doc_keys.append(d_url)

    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    return (prompt, tuple(image_keys), tuple(doc_keys), response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=1000, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    images: list[Image.Image] | list[str] | None = None,
    doc: list[str] | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> Any:
    """
    Make a chat completion request with optional images and structured output.

    Args:
        prompt: The text prompt/instruction
        images: Optional list of images to process (either PIL Images or URLs)
        response_model: Optional Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Parsed response model if response_model provided, else raw response text
    """
    content = []
    content.append({"type": "text", "text": prompt})
    if doc:
        if isinstance(doc, str):
            content.append({
                    "type": "file_url",
                    "file_url": {"url": doc, "detail": "auto"}
                })
        elif isinstance(doc, list):
            for d_url in doc:
                assert isinstance(d_url, str) and d_url.startswith("http"), "Document URLs must be strings starting with http or https"
                content.append({
                    "type": "file_url",
                    "file_url": {"url": d_url, "detail": "auto"}
                })


    if images:
        for image in images:
            if isinstance(image, str):
                assert image.startswith("http"), "Image URLs must start with http or https"
                content.append({
                    "type": "image_url",
                    "image_url": {"url": image, "detail": "auto"}
                })
            elif isinstance(image, Image.Image):
                content.append({
                    "type": "image_url",
                    "image_url": {"url": encode_image(image, format="JPEG"), "detail": "auto"}
                })
            else:
                raise ValueError("Images must be either PIL Images or URLs")

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content

    if response_model:
        return response_model.model_validate_json(response_text), response.session_id

    return response_text, response.session_id

print("Helper functions defined!")

Helper functions defined!


### 1. Full Academic Review Pass


Upload a student paper and request comprehensive feedback.

In [12]:
PAPER_URL = "https://isagraf.ru/images/industry_avt/soft/isagraf/Bata%20Shoe%20Organization-Term-Paper.pdf" 

review, session_id = chat_completion(
    prompt=(
        "You are a university teaching assistant reviewing a student's term paper. "
        "Evaluate the paper for: "
        "- Clarity and academic tone "
        "- Logical flow and argument strength "
        "- Proper citation (APA/MLA-style assumed) "
        "- Grammar, punctuation, and sentence structure "
        "- Formatting consistency (headings, margins, etc.) "
        "Return only high- and medium-severity issues with page numbers, excerpts, and specific suggestions. "
        "Conclude with an overall assessment and estimated letter grade."
    ),
    doc=PAPER_URL,
    response_model=TermPaperReview
)

print(f"📄 Paper reviewed across {len(set(f.page_number for f in review.feedback))} pages")
print(f"📊 Grade estimate: {review.grade_estimate}")
print(f"❗ Total feedback items: {review.total_feedback_items}")
print(f"📂 By category: {review.feedback_by_category}\n")
print("📝 Overall Assessment:")
print(review.overall_assessment)

📄 Paper reviewed across 8 pages
📊 Grade estimate: F
❗ Total feedback items: 8
📂 By category: {}

📝 Overall Assessment:
This document fails to meet the basic standards of an academic term paper. It is a 'Frankenstein' document composed of disparate, uncredited sources, including customer complaints and technical engineering papers. The presence of watermarks from 'free paper' websites and browser screenshots indicates a total lack of original work and severe plagiarism. Formatting is unprofessional, and the grammar is severely compromised by the incorrect use of Title Case.


### 2. Citation & Plagiarism Signal Check

Focus specifically on source integrity.

In [13]:
citation_check, _ = chat_completion(
    prompt=(
        "Scan the paper for potential academic integrity issues: "
        "- Missing in-text citations for claims or data "
        "- Over-reliance on single sources "
        "- Suspicious phrasing that may indicate uncited paraphrasing "
        "- Mismatched reference list entries "
        "Do not accuse—flag only *potential* concerns needing human review."
    ),
    doc=PAPER_URL,
    response_model=TermPaperReview
)

print(f"🔍 Citation-related flags: {len([f for f in citation_check.feedback if f.category == 'citation'])}")
for item in citation_check.feedback[:3]:
    print(f"\nPage {item.page_number} | {item.category.title()} ({item.severity})")
    print(f"Excerpt: “{item.excerpt}”")
    print(f"Suggestion: {item.suggestion}")

🔍 Citation-related flags: 3

Page 4 | Formatting (medium)
Excerpt: “... ”
Suggestion: Verify the original source of this text and ensure proper paraphrasing and attribution.

Page 8 | Citation (high)
Excerpt: “We Are Currently Investigating The Use Of Mobile Platforms... For Automatic Assembly Tasks.”
Suggestion: Attribute this technical description to the original research team or paper.

Page 9 | Formatting (high)
Excerpt: “(Russian text block follows English description of shoe components)”
Suggestion: Remove redundant foreign language text and cite the source of the technical information.


### 3. Writing Style & Tone Refinement

Help students sound more scholarly.

In [14]:
style_pass, _ = chat_completion(
    prompt=(
        "Identify informal language, vague claims, passive overuse, or weak transitions. "
        "Suggest improvements that elevate academic tone without changing meaning. "
        "Focus on undergrad-level expectations."
    ),
    doc=PAPER_URL,
    response_model=TermPaperReview
)

print(f"🖋️ Style suggestions: {len(style_pass.feedback)}")

🖋️ Style suggestions: 5


### 4. Generate Instructor Summary

Condense feedback into a grading memo.

In [16]:
class InstructorMemo(BaseModel):
    student_name: str = Field(default="Anonymous")
    paper_title: str
    strengths: List[str]
    areas_for_improvement: List[str]
    recommended_grade: str
    next_steps: str

memo, _ = chat_completion(
    prompt=(
        "Write a 5-sentence instructor memo summarizing this student paper. "
        "Highlight 2-3 strengths, 2-3 improvement areas, suggest a grade, "
        "and recommend next steps (e.g., 'revise intro', 'add 2 scholarly sources')."
    ),
    doc=PAPER_URL,
    response_model=InstructorMemo
)

print("📬 INSTRUCTOR MEMO")
print(f"Title: {memo.paper_title}")
print(f"Recommended Grade: {memo.recommended_grade}\n")
print("✅ Strengths:")
for s in memo.strengths:
    print(f"  • {s}")
print("\n🔧 Areas for Improvement:")
for a in memo.areas_for_improvement:
    print(f"  • {a}")
print(f"\n➡️ Next Steps: {memo.next_steps}")

📬 INSTRUCTOR MEMO
Title: Operations Management Analysis: Bata Shoe Organization and Bata India
Recommended Grade: B+

✅ Strengths:
  • In-depth technical analysis of manufacturing processes, including the ellipsoidal conveyor system.
  • Strong contextualization of the "Bata System" and its impact on internal cost accounting.
  • Comprehensive overview of facility planning and global supply chain logistics.

🔧 Areas for Improvement:
  • Integrate critical analysis for the raw consumer complaints to bridge data with operational solutions.
  • Increase the use of peer-reviewed scholarly sources beyond company-specific documentation.
  • Refine transitions between the technical manufacturing simulations and the broader business strategy sections.

➡️ Next Steps: Analyze consumer feedback to identify operational fixes, add 3-4 scholarly sources on management theory, and smooth the structural flow between sections.


## Conclusion

VLM Run Orion turns subjective grading into structured, scalable, and supportive feedback—ideal for:

- Teachers managing large cohorts

- Students seeking pre-submission self-review

- Writing centers offering automated first-pass analysis



### Key Takeaways

- 📑 Page-accurate feedback with direct quotes

- 🎯 Rubric-aligned categories (argument, citation, clarity, etc.)

- 💬 Constructive—not punitive—suggestions

- ⚙️ OpenAI-compatible API for easy automation



### Next Steps

- Explore the [VLM Run Documentation](https://docs.vlm.run) for more details
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support
- Check out more examples in the [VLM Run Cookbook](https://github.com/vlm-run/vlmrun-cookbook)
- Review domain-specific redaction agents for financial, healthcare, legal, and other industries

Happy building!